<a href="https://colab.research.google.com/github/bjuhasz7054/NBD_Deep_learning/blob/main/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Hyperparameters Config


In [ ]:
RANDOM_SEED = 42
TRAIN_TEST_SPLIT_RATIO = 0.8
EPOCHS = 30
TRAIN_BATCH_SIZE = 256
VALIDATE_BATCH_SIZE = 256
DECREASE_RATIO = 1
PATIENCE=5
LEARNING_RATE=0.001

# Load Data

Download dataset and labels


In [ ]:
%%capture
# download dataset
!gdown --id 1Z1RqRo0_JiavaZw2yzZG6WETdZQ8qX86
!unzip fairface-img-margin025-trainval.zip
!rm fairface-img-margin025-trainval.zip

# download labels
!gdown --id 1i1L3Yqwaio7YSOCj7ftgk8ZZchPG7dmH
!gdown --id 1wOdja-ezstMEp81tX1a-EYkFebev4h7D

Initialize dataframes

In [ ]:
import pandas as pd 

def split_dataset(base_dataframe, fraction):
  first = base_dataframe.sample(frac=fraction, random_state=RANDOM_SEED)
  second = base_dataframe.drop(first.index)
  return (first, second)

initial_train_dataset, _ =  split_dataset(
  base_dataframe=pd.read_csv(
    "fairface_label_train.csv"
  ),
  fraction=DECREASE_RATIO
)

validate_labels_df, _ = split_dataset(
  base_dataframe=pd.read_csv(
    "fairface_label_val.csv"
  ),
  fraction=DECREASE_RATIO
)

train_labels_df, test_labels_df = split_dataset(
  base_dataframe=initial_train_dataset,
  fraction=TRAIN_TEST_SPLIT_RATIO
)

In [ ]:
TRAIN_SIZE = len(train_labels_df)
VALIDATE_SIZE = len(validate_labels_df)
TEST_SIZE = len(test_labels_df)

dataset_size = TRAIN_SIZE + VALIDATE_SIZE + TEST_SIZE
print(f"train percantage = {TRAIN_SIZE / dataset_size * 100}%")
print(f"test percantage = {TEST_SIZE / dataset_size * 100}%")
print(f"validate percantage = {VALIDATE_SIZE / dataset_size * 100}%")

train percantage = 71.03011320600217%
test percantage = 17.757784192102193%
validate percantage = 11.212102601895637%


In [ ]:
train_labels_df

,file,age,gender,race,service_test
82770,train/82771.jpg,10-19,Male,White,False
80112,train/80113.jpg,40-49,Male,Latino_Hispanic,False
23469,train/23470.jpg,40-49,Female,Latino_Hispanic,True
37598,train/37599.jpg,40-49,Female,East Asian,True
63855,train/63856.jpg,40-49,Male,East Asian,True
...,...,...,...,...,...
1887,train/1888.jpg,50-59,Female,Latino_Hispanic,False
85721,train/85722.jpg,40-49,Male,East Asian,False
12363,train/12364.jpg,20-29,Male,East Asian,True
38531,train/38532.jpg,10-19,Male,Latino_Hispanic,False


In [ ]:
import tensorflow as tf
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = tf.dtypes.cast(y_true, tf.float64) * tf.dtypes.cast(K.log(y_pred), tf.float64) * tf.dtypes.cast(weights, tf.float64)
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

Preprocess Image with JPEG compression

In [ ]:
import numpy as np
import imgaug.augmenters as iaa
from PIL import Image


def jpeg_compress(np_image):
  # This function can only work with this datatype
  img_list = [np_image.astype("uint8")]
  
  # 60 - 75 means, a quality of 40 - 25
  seq_free = iaa.Sequential(
    [iaa.JpegCompression(compression=(60, 75))]
  )
   
  return seq_free(images=img_list)[0].astype("float32")

# AE model

Create Data Generators

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import random 
import numpy as np


train_datagen = ImageDataGenerator(
  rescale=1./255,
  preprocessing_function = jpeg_compress
)

validate_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

common_generator_settings = {
  "x_col": "file",
  "class_mode": 'input',
  "seed": RANDOM_SEED,
  "target_size": (224, 224),
  "validate_filenames": True
}

train_generator = train_datagen.flow_from_dataframe(
  dataframe=train_labels_df,
  batch_size=8,
  **common_generator_settings
)

test_generator = test_datagen.flow_from_dataframe(
  dataframe=test_labels_df,
  **common_generator_settings
)

validate_generator = validate_datagen.flow_from_dataframe(
  dataframe=validate_labels_df,
  batch_size=VALIDATE_BATCH_SIZE,
  **common_generator_settings
)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

Load the previously trained classification model without the Dense layers, then add a decoder network to it so it can function as an AutoEncoder.

In [ ]:
model=keras.models.load_model('/content/drive/MyDrive/figures/model.hdf5', custom_objects={'loss': weighted_categorical_crossentropy})

input=model.input

for layer in model.layers[:-5]:
  layer.trainable=False

x=model.layers[-6].output

code=x

encoder=keras.Model(input, code)

x=keras.layers.Reshape((1, 1, 512))(code)

x=keras.layers.UpSampling2D((7,7))(x)

for filter in [512]:
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.UpSampling2D((2,2))(x)

for filter in [512, 256]:
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.UpSampling2D((2,2))(x)
for filter in [128, 64]:
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.Conv2D(filter, kernel_size=3, padding='same', activation='relu')(x)
  x=keras.layers.UpSampling2D((2,2))(x)
output=keras.layers.Conv2D(3, kernel_size=3, padding='same', activation='sigmoid')(x)


ae=keras.Model(input, output)

decoder_input=keras.Input(shape=(512))
x=decoder_input
for layer in ae.layers[len(encoder.layers):]:
  x=layer(x)
decoder_output=x

decoder=keras.Model(decoder_input, decoder_output)

In [ ]:
ae.summary()
encoder.summary()
decoder.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from datetime import datetime

checkpointer=ModelCheckpoint(filepath='/content/drive/MyDrive/figures/ae_model.hdf5', save_best_only=True, monitor='loss', verbose=1)

ae.compile(optimizer='adam', loss='mse')


In [ ]:
ae.fit(
  train_generator,
  steps_per_epoch=TRAIN_SIZE//8,
  epochs=10,
  callbacks=(checkpointer)
)

Epoch 1/10
1734/1734 [==============================] - ETA: 0s - loss: 0.0361
Epoch 00001: loss improved from inf to 0.03611, saving model to /content/drive/MyDrive/figures/ae_model.hdf5
1734/1734 [==============================] - 562s 315ms/step - loss: 0.0361
Epoch 2/10
1734/1734 [==============================] - ETA: 0s - loss: 0.0288
Epoch 00002: loss improved from 0.03611 to 0.02882, saving model to /content/drive/MyDrive/figures/ae_model.hdf5
1734/1734 [==============================] - 540s 311ms/step - loss: 0.0288
Epoch 3/10
1734/1734 [==============================] - ETA: 0s - loss: 0.0270
Epoch 00003: loss improved from 0.02882 to 0.02702, saving model to /content/drive/MyDrive/figures/ae_model.hdf5
1734/1734 [==============================] - 540s 311ms/step - loss: 0.0270
Epoch 4/10
 436/1734 [======>.......................] - ETA: 6:42 - loss: 0.0264

Load 10 pictures from the dataset

In [ ]:
import glob
import tqdm
import matplotlib
x_train=np.empty((10, 224, 224, 3), dtype=np.uint8)
imgs=glob.glob("train/*.jpg")
img_count=0
for img in tqdm.tqdm(imgs):
   x_train[img_count]=matplotlib.image.imread(img)

   img_count+=1
   
   if(img_count==10):
     break

Load previously saved AutoEncoder model if necessary

In [ ]:
ae=keras.models.load_model(filepath='/content/drive/MyDrive/figures/ae_model.hdf5')
encoder=keras.Model(ae.input, ae.layers[19].output)

decoder_input=keras.Input(shape=(512))
x=decoder_input
for layer in ae.layers[len(encoder.layers):]:
  x=layer(x)
decoder_output=x

decoder=keras.Model(decoder_input, decoder_output)

Linear interpolation between two pictures using the latent space

In [ ]:
from matplotlib import animation
import tqdm
def lin_pol(x1, x2, encoder, decoder, name='proba.mp4'):
  '''Generates animation that shows the linear interpolation between samples

  Params:
    x1, x2:
      Input samples, linear interpolation will be done from x1 to x2
    encoder:
      Encoder model of the AutoEncoder network
    decoder:
      Decoder model of the AutoEncoder network
    name:
      Name of the animation file that will be saved.
  
  Usage:
    Example usage of the function:

    >>>lin_pol(x_train[0:1], x_train[8:9], encoder, decoder, name='ae_lin_pol_0_9.mp4')
    100%|██████████| 501/501 [09:51<00:00,  1.18s/it]

  '''
  fig=plt.figure(figsize=(8,8))
  ax=fig.add_subplot(111)
  code1=encoder(x1/255)
  code2=encoder(x2/255)
  code=np.zeros((1, 512))
  ims=[]
  for i in tqdm.tqdm(range(500+1)):
    for j in range(512):
      code[0][j]=code1[0][j]+(code2[0][j]-code1[0][j])/500*i
    ttl = plt.text(0.5, 1.01, str(i/5)+"% sample", horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes)
    im=plt.imshow(decoder(code)[0])
    ims.append([im, ttl])
  ani=animation.ArtistAnimation(fig, ims, blit=True, interval=50)
  ani.save("/content/drive/MyDrive/figures/"+name)

In [ ]:
plt.imshow(x_train[0])
plt.show()
plt.imshow(x_train[9])

In [ ]:
lin_pol(x_train[0:1], x_train[8:9], encoder, decoder, name='ae_lin_pol_0_9.mp4')